# Setup

In [ ]:
import pandas as pd
import PIL
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as XLImage 
from PIL import Image as PILImage
import shutil
import os
import zipfile
import json

# Define inputs

In [ ]:
# Imput request date YYY-MM-DD
request_date = '2024-11-08'

# Imput email column
input_file_email_column = 'Email'

# imput file path definition
input_file_path = request_date + '/' + request_date + '_request.xlsx'

# Users data file path definition
data_path = request_date + '/' + request_date + '_users_data.xlsx'

# Output directory definition
output_folder = request_date + '/' + request_date + '_output'

# Base files
logo_path = "mc_logo.png"
template_path = "irm template.xlsx"


In [ ]:
# Create new directory if needed
os.makedirs(request_date, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Define functions

In [ ]:
# Function that generates a query

def generate_query(input_file_path, email_column='Email'):
    # Base query
    base_query = """--query used for the requests of user single data
SELECT
profileid AS`ProfileId`,
first_name AS`First Name`,
last_name AS`Last Name`,
normalized_name AS`Normalized Name`,
date(FROM_UNIXTIME(birth_date / 1000)) AS`Birthdate`,
email AS`Primary Email Address`,
email_addresses_all AS`Email Addresses (all)`,
REPLACE(primary_phone, '+', '') AS`Primary Phone`,
REPLACE(phone_number, '+', '') AS`Phone Numbers`,
REPLACE(phone_numbers_all, '+', '') AS`Phone Numbers (all)`,
REPLACE(shopify_phone, '+', '') AS`Shopify Phone`,
email_campaign_id_all AS`Email CampaignId (all)`,
email_id_all AS`EmailId (all)`,
city AS`City (all)`,
mr_geo_city_name AS`City (mostrecent)`,
geo_city_name AS`Cityname`,
geo_subdivision_1_name AS`State (all)`,
mr_geo_subdivision_1_name AS`State (mostrecent)`,
geo_subdivision_1_iso_code AS`Statecode (all)`,
mr_geo_subdivision_1_iso_code AS`Statecode (mostrecent)`,
geo_subdivision_2_name AS`County (all)`,
mr_geo_subdivision_2_name AS`County (mostrecent)`,
geo_subdivision_2_iso_code AS`Countycode (all)`,
mr_geo_subdivision_2_iso_code AS`Countycode (mostrecent)`,
geo_continent_code AS`Continentcode (all)`,
geo_continent_name AS`Continent (all)`,
geo_country_iso_code AS`Countrycode`,
country AS`Country (all)`,
geo_metro_code AS`DesignatedMarketAreacode (all)`,
mr_geo_metro_code AS`Designated Market Area code (mostrecent)`,
geo_geoname_id AS`GeonameID (all)`, 
mr_geo_geoname_id AS`GeonameID (mostrecent)`,
geo_latlong AS`Coordinates (all)`, 
geo_time_zone AS`Timezone (all)`,
mr_geo_time_zone AS`Timezone (mostrecent)`,
geo_zipcode AS`Zipcode`,
mr_geo_zipcode AS`Zipcode (mostrecent)`,
postal_code AS`Postalcode`,
language AS`Language`,
experian_ethnic_religion AS`Experian Ethnic Religion`,
experian_gender AS`Experian Gender`
from prod_products.cdp_reporting.bcexport_allconsumers
WHERE email in (
{email_list}
)"""

    # Read Excel file
    df = pd.read_excel(input_file_path)
    
    # Extract emails, convert to lowercase, and format them
    emails = df[email_column].str.lower().tolist()
    formatted_emails = ',\n'.join(f'"{email}"' for email in emails)
    
    # Insert formatted emails into query
    final_query = base_query.format(email_list=formatted_emails)
    
    return final_query

In [ ]:
def check_emails(source1, source2, email_col1='Primary Email Address', email_col2='Primary Email Address'):
    """
    Compares emails between two sources (Excel files or pandas DataFrames).
    
    Parameters:
    - source1: first source (path to Excel file or pandas DataFrame)
    - source2: second source (path to Excel file or pandas DataFrame)
    - email_col1: email column name in source 1
    - email_col2: email column name in source 2
    
    Returns:
    - List of emails present in source1 but not in source2
    """
    
    # Handle different input types for source1
    if isinstance(source1, str):
        df1 = pd.read_excel(source1)
    elif isinstance(source1, pd.DataFrame):
        df1 = source1.copy()
    else:
        raise ValueError("source1 must be either a file path (str) or a pandas DataFrame")

    # Handle different input types for source2
    if isinstance(source2, str):
        df2 = pd.read_excel(source2)
    elif isinstance(source2, pd.DataFrame):
        df2 = source2.copy()
    else:
        raise ValueError("source2 must be either a file path (str) or a pandas DataFrame")
    
    # Convert emails to lowercase and remove whitespace
    df1[email_col1] = df1[email_col1].str.lower().str.strip()
    df2[email_col2] = df2[email_col2].str.lower().str.strip()
    
    # Create email sets
    emails1 = set(df1[email_col1].dropna())
    emails2 = set(df2[email_col2].dropna())
    
    # Find emails not present in source 2
    missing_emails = list(emails1 - emails2)
    
    # Print statistics
    print(f"\nTotal emails in source 1: {len(emails1)}")
    print(f"Total emails in source 2: {len(emails2)}")
    print(f"Emails not found: {len(missing_emails)}")
    print(f"Not found percentage: {(len(missing_emails)/len(emails1))*100:.2f}%")
    
    return missing_emails

In [ ]:
def process_and_generate_files(request_date, template_path, excel_path, json_folder, output_folder, logo_path, properties_mapping):
    """
    Función combinada que procesa datos de Excel y JSON, y genera archivos individuales
    """
    # 1. Cargar datos del Excel inicial
    df_excel = pd.read_excel(excel_path)
    
    # 2. Procesar archivos JSON con todas las transformaciones
    all_data = []
    for filename in os.listdir(json_folder):
        if filename.endswith('.json'):
            file_path = os.path.join(json_folder, filename)
            with open(file_path, 'r') as file:
                data = json.load(file)
            
            row_data = {
                'Creation Date': data['creationDate'],
                'ID': data['id']
            }
            
            # Inicializar con None
            for old_name, new_name in properties_mapping.items():
                row_data[new_name] = None
            
            # Procesar propiedades con transformaciones
            for prop in data['properties']['property']:
                if prop['id'] in properties_mapping:
                    value = prop['values'][0] if prop['values'] else None
                    
                    # Aplicar transformaciones específicas
                    if value is not None:
                        if prop['id'] in ['primary_phone', 'phone_number', 'phone_numbers_all', 'shopify_phone']:
                            value = value.replace('+', '')
                        elif prop['id'] == 'birth_date':
                            try:
                                timestamp = int(value) / 1000
                                value = pd.to_datetime(timestamp, unit='s').strftime('%Y-%m-%d')
                            except (ValueError, TypeError):
                                value = None
                    
                    row_data[properties_mapping[prop['id']]] = value
            
            all_data.append(row_data)
    
    # Crear DataFrame de JSON
    df_json = pd.DataFrame(all_data)
    
    # 3. Concatenar DataFrames (apilar filas)
    df_combined = pd.concat([df_excel, df_json], ignore_index=True)
    
    # 4. Crear archivos individuales y ZIP
    os.makedirs(output_folder, exist_ok=True)
    created_files = []
    
    # Iterar sobre el DataFrame combinado
    for _, row in df_combined.iterrows():
        email = row['Primary Email Address']
        if pd.isna(email):
            continue
            
        output_path = f"{output_folder}/{email}.xlsx"
        shutil.copyfile(template_path, output_path)
        
        # Modificar archivo Excel
        wb = load_workbook(output_path)
        ws = wb.active
        
        # Insertar datos en fila 4
        for col_idx, value in enumerate(row, start=1):
            ws.cell(row=4, column=col_idx, value=value)
        
        # Añadir logo
        img = XLImage(logo_path)
        ws.add_image(img, "A1")
        
        wb.save(output_path)
        print(f"File saved: {output_path}")
        created_files.append(output_path)
    
    # Crear archivo ZIP
    zip_path = f"{output_folder}/{request_date}.zip"
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for file in created_files:
            zipf.write(file, os.path.basename(file))
    
    print(f"ZIP file created: {zip_path}")
    
    return df_combined

# Create databricks query

In [ ]:
query = generate_query(input_file_path, email_column=input_file_email_column)
print(query)

# Check users

In [ ]:
# Function usage:
missing_df = check_emails(
    input_file_path,
    data_path,
    email_col1=input_file_email_column,  # Adjust according to column names
    email_col2='Primary Email Address'
)

# View not found emails
print("\nNot found emails:")
print(missing_df)

# Create users data files

In [ ]:
# 1. Primero definir el properties_mapping
properties_mapping = {
    'profileid': 'ProfileId',
    'first_name': 'First Name',
    'last_name': 'Last Name',
    'normalized_name': 'Normalized Name',
    'birth_date': 'Birthdate',
    'email': 'Primary Email Address',
    'email_addresses_all': 'Email Addresses (all)',
    'primary_phone': 'Primary Phone',
    'phone_number': 'Phone Numbers',
    'phone_numbers_all': 'Phone Numbers (all)',
    'shopify_phone': 'Shopify Phone',
    'email_campaign_id_all': 'Email CampaignId (all)',
    'email_id_all': 'EmailId (all)',
    'city': 'City (all)',
    'mr_geo_city_name': 'City (mostrecent)',
    'geo_city_name': 'Cityname',
    'geo_subdivision_1_name': 'State (all)',
    'mr_geo_subdivision_1_name': 'State (mostrecent)',
    'geo_subdivision_1_iso_code': 'Statecode (all)',
    'mr_geo_subdivision_1_iso_code': 'Statecode (mostrecent)',
    'geo_subdivision_2_name': 'County (all)',
    'mr_geo_subdivision_2_name': 'County (mostrecent)',
    'geo_subdivision_2_iso_code': 'Countycode (all)',
    'mr_geo_subdivision_2_iso_code': 'Countycode (mostrecent)',
    'geo_continent_code': 'Continentcode (all)',
    'geo_continent_name': 'Continent (all)',
    'geo_country_iso_code': 'Countrycode',
    'country': 'Country (all)',
    'geo_metro_code': 'DesignatedMarketAreacode (all)',
    'mr_geo_metro_code': 'Designated Market Area code (mostrecent)',
    'geo_geoname_id': 'GeonameID (all)',
    'mr_geo_geoname_id': 'GeonameID (mostrecent)',
    'geo_latlong': 'Coordinates (all)',
    'geo_time_zone': 'Timezone (all)',
    'mr_geo_time_zone': 'Timezone (mostrecent)',
    'geo_zipcode': 'Zipcode',
    'mr_geo_zipcode': 'Zipcode (mostrecent)',
    'postal_code': 'Postalcode',
    'language': 'Language',
    'experian_ethnic_religion': 'Experian Ethnic Religion',
    'experian_gender': 'Experian Gender'
}

# 2. Llamar a la función
df_final = process_and_generate_files(
    request_date=request_date,                          # Fecha para nombrar el ZIP
    template_path='irm template.xlsx',         # Ruta de la plantilla Excel
    excel_path=request_date + '/' + request_date + '_users_data.xlsx',            # Ruta del Excel inicial
    json_folder= request_date,                 # Carpeta con archivos JSON
    output_folder= request_date + '/' + request_date + '_output',        # Carpeta donde se guardarán los archivos
    logo_path= 'mc_logo.png',                    # Ruta del logo
    properties_mapping= properties_mapping               # Diccionario de mapeo definido arriba
)

# 3. Opcionalmente, revisar el DataFrame resultante
print(df_final.shape)  # Ver dimensiones del DataFrame
print(df_final.columns)  # Ver columnas

# Final check

In [ ]:
# Function usage:
missing_df = check_emails(
    input_file_path,
    df_final,
    email_col1=input_file_email_column,  # Adjust according to column names
    email_col2='Primary Email Address'
)

# View not found emails
print("\nNot found emails:")
print(missing_df)